# Products Clusters

In [1]:
# Import packages
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Autoreload
%load_ext autoreload
%autoreload 2

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Import data
from bimbo.data import Bimbo

data = Bimbo().get_data()

# Load in the train and test datasets
# train_df = data['train']
# test_df = data['test']
# cliente_tabla_df = data['cliente_tabla']
producto_tabla_df = data['producto_tabla']
# town_state_df = data['town_state']
# sample_submission_df = data['sample_submission']

Memory usage of dataframe is 0.04 MB
Memory usage after optimization is: 0.11 MB
Decreased by -184.7%
Memory usage of dataframe is 14.27 MB
Memory usage after optimization is: 19.51 MB
Decreased by -36.7%
Memory usage of dataframe is 373.80 MB
Memory usage after optimization is: 120.15 MB
Decreased by 67.9%
Memory usage of dataframe is 0.02 MB
Memory usage after optimization is: 0.02 MB
Decreased by 6.0%
Memory usage of dataframe is 6225.47 MB
Memory usage after optimization is: 2122.32 MB
Decreased by 65.9%
Memory usage of dataframe is 106.80 MB
Memory usage after optimization is: 33.38 MB
Decreased by 68.7%


In [2]:
producto_tabla_df.head(3)

,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41


### Text pre-processing

In [3]:
from bimbo.text_preprocess import Preprocess

In [4]:
text = "Capuccino Moka 750g NES 9"

In [5]:
pr = Preprocess()

In [6]:
lower_text = pr.to_lower(text)
sentence_tokens = pr.sentence_tokenize(lower_text)
for each_sent in sentence_tokens:
    lemmatizzed_sent = pr.lemmatize(each_sent)
    clean_text = pr.remove_numbers(lemmatizzed_sent)
    clean_text = pr.remove_punct(clean_text)
    clean_text = pr.remove_Tags(clean_text)
    clean_text = pr.remove_stopwords(clean_text)
    word_tokens = pr.word_tokenize(clean_text)
    print(word_tokens)

['capuccino', 'moka', 'g', 'ne']


In [7]:
print(producto_tabla_df['NombreProducto'])

0                                    NO IDENTIFICADO 0
1                            Capuccino Moka 750g NES 9
2               Bimbollos Ext sAjonjoli 6p 480g BIM 41
3                       Burritos Sincro 170g CU LON 53
4                  Div Tira Mini Doradita 4p 45g TR 72
5                    Pan Multigrano Linaza 540g BIM 73
6                         Tostado Integral 180g WON 98
7                               Pan Blanco 567g WON 99
8               Super Pan Bco Ajonjoli 680g SP WON 100
9                   Wonder 100pct mediano 475g WON 106
10                   Wonder 100pct gde 680g SP WON 107
11                 Baguette Precocida Cong 280g DH 108
12                        Pan Multicereal 475g WON 109
13                       Tostado Integral 180g WON 112
14            Biscotel Receta Original 410g CU SUA 122
15                    Super Bollos 5in 8p 540g WON 123
16                              Bollos 8p 450g WON 125
17                      Bollos BK 4in 36p 1635g SL 131
18        

In [8]:
# Found on Kaggle (https://www.kaggle.com/vykhand/exploring-products)
# Credit: Andrey Vykhodtsev

producto_tabla_df['short_name'] = producto_tabla_df.NombreProducto.str.extract('^(\D*)', expand=False)
producto_tabla_df['brand'] = producto_tabla_df.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = producto_tabla_df.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
producto_tabla_df['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
producto_tabla_df['pieces'] =  producto_tabla_df.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
producto_tabla_df['short_name_and_brand'] = producto_tabla_df['short_name']  + producto_tabla_df['brand']
producto_tabla_df.head()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces,short_name_and_brand
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN,NO IDENTIFICADO IDENTIFICADO
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750.0,NaN,Capuccino Moka NES
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480.0,6.0,Bimbollos Ext sAjonjoli BIM
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170.0,NaN,Burritos Sincro LON
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45.0,4.0,Div Tira Mini Doradita TR


In [9]:
producto_tabla_df.short_name.value_counts(dropna=False)

Pan Blanco                          39
Tortillinas                         37
Deliciosas Vainilla                 35
Medias Noches                       25
Wonder                              22
Pan Integral                        19
Canelitas                           17
Suavicremas Chocolate               17
Deliciosas Chochitos                17
Suavicremas Vainilla                17
Choco Roles                         16
Suavicremas Fresa                   16
Chocochispas                        15
Mantecadas Nuez                     15
Tostada Ondulada Tubo               15
Canapinas                           15
Tostado                             14
Barritas Fresa                      14
Barritas Pina                       13
Bran Frut Fresa                     12
Plativolos                          12
Totopo Chilaquiles                  12
Triki Trakes                        12
Bollos BK                           12
Barra Multigrano Nuez               12
Deliciosas Chocochispas  

In [10]:
len(producto_tabla_df['short_name'].value_counts()[producto_tabla_df['short_name'].value_counts()>0])

1014

In [11]:
producto_tabla_df["ProcessedText"] = producto_tabla_df.short_name_and_brand.apply(lambda x : pr.preprocess(f"{x}"))
producto_tabla_df["ProcessedText"]

0              [identificado, identificado]
1                     [capuccino, moka, ne]
2          [bimbollos, ext, sajonjoli, bim]
3                    [burrito, sincro, lon]
4           [div, tira, mini, doradita, tr]
5            [pan, multigrano, linaza, bim]
6                  [tostado, integral, won]
7                        [pan, blanco, won]
8          [super, pan, bco, ajonjoli, won]
9                             [wonder, won]
10                            [wonder, won]
11          [baguette, precocida, cong, dh]
12                  [pan, multicereal, won]
13                 [tostado, integral, won]
14        [biscotel, receta, original, sua]
15                     [super, bollos, won]
16                            [bollos, won]
17                         [bollos, bk, sl]
18                         [bollos, bk, sl]
19                         [bollos, bk, sl]
20                         [bollos, bk, sl]
21                          [hot, dog, won]
22                   [super, hot

In [12]:
# # SHOULD I ADD MORE STOP WORDS ???? (and take them away ????)
# stop = ['g','kg']

# producto_tabla_df["ProcessedText"] = producto_tabla_df["ProcessedText"]\
#                                     .apply(lambda x: [item for item in x if item not in stop])
# producto_tabla_df["ProcessedText"]

### product clustering

In [13]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

vocab_size = 10000 # maximum number of words to keep (keeps the most frequent)
maxlen = 200 # maximum number of words in a review

Using TensorFlow backend.


In [14]:
tokenizer = Tokenizer(num_words=maxlen)
tokenizer.fit_on_texts(list(producto_tabla_df["ProcessedText"].values))
X_train = tokenizer.texts_to_sequences(producto_tabla_df["ProcessedText"] .values)
x_train = sequence.pad_sequences(X_train, maxlen = maxlen)

In [15]:
x_train

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,  41,   1],
       ...,
       [  0,   0,   0, ...,  35, 163,   8],
       [  0,   0,   0, ...,  35, 163,   8],
       [  0,   0,   0, ...,  35, 163,   8]], dtype=int32)

In [16]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30) 
# because 98 = len(producto_tabla_df['short_name'].value_counts()[producto_tabla_df['short_name'].value_counts()>5])
kmeans.fit(x_train)
y_kmeans = kmeans.predict(x_train)

In [17]:
y_kmeans

array([ 9,  9,  1, ..., 18, 18, 18], dtype=int32)

In [18]:
np.unique(y_kmeans)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int32)

In [19]:
cluster_df = pd.DataFrame(y_kmeans, columns=['cluster'])
cluster_df.head(3)

,cluster
0,9
1,9
2,1


In [20]:
producto_clusters_df = pd.concat([producto_tabla_df, cluster_df + 1], axis=1)
producto_clusters_df

,Producto_ID,NombreProducto,short_name,brand,weight,pieces,short_name_and_brand,ProcessedText,cluster
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN,NO IDENTIFICADO IDENTIFICADO,"[identificado, identificado]",10
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750.0,NaN,Capuccino Moka NES,"[capuccino, moka, ne]",10
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480.0,6.0,Bimbollos Ext sAjonjoli BIM,"[bimbollos, ext, sajonjoli, bim]",2
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170.0,NaN,Burritos Sincro LON,"[burrito, sincro, lon]",1
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45.0,4.0,Div Tira Mini Doradita TR,"[div, tira, mini, doradita, tr]",14
5,73,Pan Multigrano Linaza 540g BIM 73,Pan Multigrano Linaza,BIM,540.0,NaN,Pan Multigrano Linaza BIM,"[pan, multigrano, linaza, bim]",15
6,98,Tostado Integral 180g WON 98,Tostado Integral,WON,180.0,NaN,Tostado Integral WON,"[tostado, integral, won]",11
7,99,Pan Blanco 567g WON 99,Pan Blanco,WON,567.0,NaN,Pan Blanco WON,"[pan, blanco, won]",10
8,100,Super Pan Bco Ajonjoli 680g SP WON 100,Super Pan Bco Ajonjoli,WON,680.0,NaN,Super Pan Bco Ajonjoli WON,"[super, pan, bco, ajonjoli, won]",7
9,106,Wonder 100pct mediano 475g WON 106,Wonder,WON,475.0,NaN,Wonder WON,"[wonder, won]",15


In [21]:
producto_clusters_df['cluster'].value_counts(dropna=False)

10    470
15    250
2     242
11    236
1     216
19    121
12    114
21     98
20     96
30     92
23     80
18     76
3      72
16     65
5      62
17     53
9      40
7      28
13     27
27     25
26     22
6      21
29     18
4      16
14     15
28     13
8      10
24      7
22      4
25      3
Name: cluster, dtype: int64

In [22]:
producto_clusters_df[producto_clusters_df['cluster'] == 19]

,Producto_ID,NombreProducto,short_name,brand,weight,pieces,short_name_and_brand,ProcessedText,cluster
29,179,Tostado Int 0pct Grasa Azucar 200g WON 179,Tostado Int,WON,200.0,NaN,Tostado Int WON,"[tostado, int, won]",19
51,634,Donitas Espolvoreadas ME 12p 210g BIM 634,Donitas Espolvoreadas ME,BIM,210.0,12.0,Donitas Espolvoreadas ME BIM,"[donitas, espolvoreadas, bim]",19
71,1101,Tortillinas Gde McD 500g Cn TR 1101,Tortillinas Gde McD,TR,500.0,NaN,Tortillinas Gde McD TR,"[tortillinas, gde, mcd, tr]",19
78,1126,Tortillinas Grande 10p 510g TR 1126,Tortillinas Grande,TR,510.0,10.0,Tortillinas Grande TR,"[tortillinas, grande, tr]",19
82,1143,Pan Integral Gde 2pq 1360g BIM 1143,Pan Integral Gde,BIM,1360.0,NaN,Pan Integral Gde BIM,"[pan, integral, gde, bim]",19
111,1242,Donitas Espolvoreadas 6p 105g BIM 1242,Donitas Espolvoreadas,BIM,105.0,6.0,Donitas Espolvoreadas BIM,"[donitas, espolvoreadas, bim]",19
124,1554,Bimbollos con Ajn 18p 1440g BIM 1554,Bimbollos con Ajn,BIM,1440.0,18.0,Bimbollos con Ajn BIM,"[bimbollos, ajn, bim]",19
196,3535,Decanelas 100g LAR 3535,Decanelas,LAR,100.0,NaN,Decanelas LAR,"[decanelas, lar]",19
238,4304,Mantecadas Chocolate SP 2p 95g TR 4304,Mantecadas Chocolate SP,TR,95.0,2.0,Mantecadas Chocolate SP TR,"[mantecadas, chocolate, sp, tr]",19
379,30091,Raztachoc 1p 40g BIM 30091,Raztachoc,BIM,40.0,1.0,Raztachoc BIM,"[raztachoc, bim]",19


In [23]:
producto_clusters_df['Producto_ID'].nunique()

2592

In [24]:
producto_clusters_df['short_name'].nunique()

1014

In [25]:
producto_clusters_df['brand'].nunique()

44

We are going to keep `short_name` and `brand`. Keeping the clusters don't make enough sense.

In [26]:
producto_clusters_df = producto_clusters_df[['Producto_ID', 'short_name', 'brand']]

### Write CSV

In [27]:
producto_clusters_df.to_csv('../data/producto_clusters.csv',index=False)